In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
training_data = pd.read_csv('/tmp/google/training_set.csv')
training_data.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2018-02-20,54.528500,55.697498,54.425999,55.123001,55.123001,28462000
1,2018-02-21,55.323502,56.698502,55.316502,55.567001,55.567001,30258000
2,2018-02-22,55.809502,56.140999,55.129501,55.331501,55.331501,26344000
3,2018-02-23,55.632000,56.363998,55.235699,56.339500,56.339500,25220000
4,2018-02-26,56.389999,57.198002,56.334751,57.187500,57.187500,31182000


In [3]:
training_data.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
1240,2023-01-24,99.550003,101.089996,98.699997,99.209999,99.209999,27391400
1241,2023-01-25,97.199997,97.720001,95.262001,96.730003,96.730003,31000900
1242,2023-01-26,98.279999,99.209999,96.820000,99.160004,99.160004,24542100
1243,2023-01-27,99.050003,101.580002,98.970001,100.709999,100.709999,29011900
1244,2023-01-30,98.745003,99.408997,97.519997,97.949997,97.949997,24365100


In [4]:
training_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1245 entries, 0 to 1244
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       1245 non-null   object 
 1   Open       1245 non-null   float64
 2   High       1245 non-null   float64
 3   Low        1245 non-null   float64
 4   Close      1245 non-null   float64
 5   Adj Close  1245 non-null   float64
 6   Volume     1245 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 68.2+ KB


In [5]:
training_set = training_data.iloc[:, 1:2].values

In [6]:
training_set.shape, training_data.shape

((1245, 1), (1245, 7))

In [13]:
training_set

array([[54.5285  ],
       [55.323502],
       [55.809502],
       ...,
       [98.279999],
       [99.050003],
       [98.745003]])

In [7]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0, 1))
training_set_scaled = sc.fit_transform(training_set)

In [8]:
training_set_scaled

array([[0.05654342],
       [0.06424928],
       [0.06896003],
       ...,
       [0.48062152],
       [0.48808508],
       [0.48512875]])

In [12]:
training_set_scaled.shape

(1245, 1)

In [10]:
x_train = []
y_train = []

for i in range(60, 1245):
    x_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0])
    
x_train, y_train = np.array(x_train), np.array(y_train)



In [11]:
x_train

array([[0.05654342, 0.06424928, 0.06896003, ..., 0.0580119 , 0.06111362,
        0.05626718],
       [0.06424928, 0.06896003, 0.06723952, ..., 0.06111362, 0.05626718,
        0.05011705],
       [0.06896003, 0.06723952, 0.07458671, ..., 0.05626718, 0.05011705,
        0.05136742],
       ...,
       [0.42488745, 0.45638932, 0.45454764, ..., 0.48886045, 0.49293152,
        0.47015319],
       [0.45638932, 0.45454764, 0.40918503, ..., 0.49293152, 0.47015319,
        0.48062152],
       [0.45454764, 0.40918503, 0.364937  , ..., 0.47015319, 0.48062152,
        0.48808508]])

In [14]:
y_train

array([0.05011705, 0.05136742, 0.04262928, ..., 0.48062152, 0.48808508,
       0.48512875])

In [15]:
x_train.shape

(1185, 60)

In [16]:
x_train = x_train.reshape(1185, 60, 1)

In [17]:
x_train.shape

(1185, 60, 1)

In [19]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(units=60, activation='relu', return_sequences=True, input_shape=(60, 1)))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.LSTM(units=60, activation='relu', return_sequences=True))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.LSTM(units=80, activation='relu', return_sequences=True))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.LSTM(units=120, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(units=1))

In [20]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 60, 60)            14880     
_________________________________________________________________
dropout (Dropout)            (None, 60, 60)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 60, 60)            29040     
_________________________________________________________________
dropout_1 (Dropout)          (None, 60, 60)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 60, 80)            45120     
_________________________________________________________________
dropout_2 (Dropout)          (None, 60, 80)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 120)              

In [21]:
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, batch_size=32, epochs=100)

Epoch 1/100
38/38 [==============================] - 24s 478ms/step - loss: 0.0681
Epoch 2/100
38/38 [==============================] - 18s 481ms/step - loss: 0.0077
Epoch 3/100
38/38 [==============================] - 18s 479ms/step - loss: 0.0055
Epoch 4/100
38/38 [==============================] - 18s 484ms/step - loss: 0.0054
Epoch 5/100
38/38 [==============================] - 18s 478ms/step - loss: 0.0052
Epoch 6/100
38/38 [==============================] - 18s 484ms/step - loss: 0.0050
Epoch 7/100
38/38 [==============================] - 18s 479ms/step - loss: 0.0055
Epoch 8/100
38/38 [==============================] - 18s 478ms/step - loss: 0.0058
Epoch 9/100
38/38 [==============================] - 18s 474ms/step - loss: 0.0045
Epoch 10/100
38/38 [==============================] - 18s 484ms/step - loss: 0.0062
Epoch 11/100
38/38 [==============================] - 18s 480ms/step - loss: 0.0048
Epoch 12/100
38/38 [==============================] - 18s 474ms/step - loss: 0.0039
E

38/38 [==============================] - 21s 550ms/step - loss: 0.0017
Epoch 99/100
38/38 [==============================] - 20s 527ms/step - loss: 0.0019
Epoch 100/100
38/38 [==============================] - 21s 539ms/step - loss: 0.0018


In [22]:
test_data = pd.read_csv('/tmp/google/test_set.csv')
test_data.shape

(19, 7)

In [23]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       19 non-null     object 
 1   Open       19 non-null     float64
 2   High       19 non-null     float64
 3   Low        19 non-null     float64
 4   Close      19 non-null     float64
 5   Adj Close  19 non-null     float64
 6   Volume     19 non-null     int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 1.2+ KB


In [24]:
real_stock_price = test_data.iloc[:, 1:2].values

In [25]:
real_stock_price

array([[89.830002],
       [91.010002],
       [88.07    ],
       [87.360001],
       [89.195   ],
       [86.720001],
       [90.059998],
       [92.400002],
       [91.528   ],
       [92.779999],
       [92.940002],
       [91.389999],
       [95.949997],
       [99.129997],
       [99.550003],
       [97.199997],
       [98.279999],
       [99.050003],
       [98.745003]])

In [26]:
real_stock_price.shape

(19, 1)

In [ ]:
dataset_total = pd.concat((training_data['Open'], test_data['Open']), axis=0)
inputs = dataset_total[len(dataset_total)-len(test_data)-60:].values
inputs = inputs.reshape(-1, 1)
inputs = sc.transform(inputs)

x_test = []
for i in range(60, 80):
    x_test.append(inputs[i-60:i, 0])
    
x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

predicted_stock_price = model.predict(x_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

In [33]:
print(predicted_stock_price[1]), print(real_stock_price[1])

[97.16311]
[91.010002]


(None, None)